In [88]:
import torch
import pandas as pd
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score,confusion_matrix
from imblearn.combine import SMOTETomek
overSampler=SMOTETomek()

from sklearn.model_selection import train_test_split


import torch.nn as nn
from torch.utils.data import DataLoader,Dataset

In [89]:
data=pd.read_parquet('/home/hashmi/Files/DataFolder/ParticleBoxData/DownstreamData.parquet')

In [90]:
data=data[data['track_p']<100e3]
data.track_nLHCbIDs=data.track_nLHCbIDs.astype('int')


sample=data.sample(frac=0.01,random_state=46)

x=sample.drop(['Downstream',],axis=1)
y=sample['Downstream'].map({True:1,False:0})

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()

x=ss.fit_transform(x)


In [91]:
x_train,x_test,y_train,y_test=train_test_split(x,y,shuffle=True,stratify=y,random_state=29,test_size=0.3)

x_train,y_train=overSampler.fit_resample(X=x_train,y=y_train)

In [92]:
class DataFun(Dataset):
    def __init__(self,x_data,y_data):
        self.x_data=x_data
        self.y_data=y_data
    def __getitem__(self,index):
        return self.x_data[index],self.y_data[index]
    def __len__(self):
        return len(self.x_data)

In [100]:
train_data=DataFun(torch.FloatTensor(x_train),torch.FloatTensor(y_train))

In [101]:
train_loader=DataLoader(train_data,batch_size=4,shuffle=True)

In [102]:
class Network(nn.Module):
    def __init__(self,input_num,output_num):
        super(Network,self).__init__()
        self.layer1=nn.Linear(input_num,15)
        self.relu1=nn.ReLU()
        self.layer2=nn.Linear(15,output_num)
        self.sigmoid=nn.Sigmoid()

    def forward(self,x):
        output = self.layer1(x)
        output = self.relu1(output)
        output = self.layer2(output)
        output = self.sigmoid(output)
        return output

In [103]:
model = Network(12,1)

In [147]:
epoch = 10
lr=0.01

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCEWithLogitsLoss()

# optimizer = torch.optim.Adam(params=model.parameters())
optimizer = torch.optim.SGD(params=model.parameters(),lr=lr)

In [148]:
for epoch in range(1,epoch+1):

    epoch_loss = 0

    for x_batch,y_batch in train_loader:
        optimizer.zero_grad()
        y_predict = model(x_batch).detach()
        loss = criterion(y_batch.unsqueeze(1),y_predict)
        loss.backward(requires_grad=True)
        optimizer.step()
        # epoch_loss+=loss.item()
        # print(y_batch.unsqueeze(1))
        # print(y_predict.unsqueeze(1))
        # print(loss)
        break
    break
    # print(epoch,epoch_loss)

TypeError: backward() got an unexpected keyword argument 'requires_grad'